# Data API for Statistics Denmark (DST)

This notebook provides a short demo on how to call the DST api. 

Another approach for Python is found in Kristian Urup Larsen's repository [here](https://github.com/Kristianuruplarsen/PyDST) that implements functions for pulling data. 

The notebook contains a function that takes as input a table code and certain input parameters. The output is a Pandas DataFrame. The notebook provides an example of how to apply the function and structure the results.

In [1]:
import requests
from io import StringIO
from copy import deepcopy

import pandas as pd

dst_api_base = 'https://api.statbank.dk/v1/data/%s/csv?'
options_base =  {'valuePresentation':'Default',
                'timeOrder':'Ascending',
                'allowVariablesInHead':'true'}

def dst_api_dataframe(dataset, options_input={}, print_url=True):    
    '''
    This function produces a dataframe for the desired 
    Statistics Denmark (DST) table. It can be specified 
    to pull particular columns.
    
    Parameters
    ----------
    dataset : str
        DST name/code table to fetch
    options_input : dict, optional
        The extra options can be speficied to pull particular
        column values. E.g. {'ABC':'*'} will return all values 
        of the column 'ABC'. Note specific column values 
        requires separation with comma_char, see example below.
        
    Returns    
    -------
    df_out : pandas.DataFrame
        Table with desired formatted as a pandas DataFrame.
    '''
    
    url = dst_api_base % dataset
    
    options = deepcopy(options_base)
    
    for k, v in options_input.items():
        options[k] = v
    
    for k, v in options.items():
        url += '%s=%s&' % (k,v) 
        
    if print_url: print(url[:-1])
        
    response = requests.get(url[:-1])
    
    df_out = pd.read_csv(StringIO(response.text), sep=';') 
    
    return df_out 
    


# Example of calling API


### Parish level
In the following example we call the API using to fetch the table `KM5` which contain parish level demographic data. We want to pull all parishes for the years 2008-2010 and age 18-65.

In [4]:
area = '*' # all parish
ages = '*'
time = range(2010, 2015)

comma_char = '%2C' # separation character for making url

request_options =  {'SOGN': area, 
                    'TID': comma_char.join(map(str, time)),
                    'ALDER': ages}
    
dst_data = dst_api_dataframe('KM5', options_input=request_options)

https://api.statbank.dk/v1/data/KM5/csv?valuePresentation=Default&timeOrder=Ascending&allowVariablesInHead=true&SOGN=*&TID=2010%2C2011%2C2012%2C2013%2C2014


Basic structuring of the dataset which includes renaming to english names. We preserves the codes for  parish for easy joining with other datasets. Note `SOGNEKODE` means parish code.

In [97]:
dst_data['age_lb'] = dst_data.ALDER.str.findall("[0-9]*").str[0].astype(int)

dst_data.drop('ALDER', axis=1, inplace=True)

col_map = {'TID':'year', 
           'INDHOLD':'obs_value'}

dst_data['kode'] = dst_data.SOGN.str[:4]
dst_data.rename(columns=col_map, inplace=True)

***Structuring the data***

We compute the share of young people in each parish.

In [98]:
# define youth variable
dst_data['young'] = dst_data.age_lb.between(20,29)
c_share = 'Share of young (20-30)'
c_count = 'Count of young (20-30)'

# count number of young using groupby
young_count = dst_data\
                .groupby(['kode', 'year', 'young'])\
                .obs_value.sum()\
                .unstack(level=2)\
        

young_share = young_count\
                .pipe(lambda df: df[True]/df.sum(1))\
                .rename(c_share)\
    
    
youth_stats = pd.concat([young_share, 
                         young_count[True].rename(c_count)], 1)\
                .pipe(lambda df: df.mask(df.isnull().max(1)).reset_index())

### Municipal level

In [99]:
area = '*' # all parish
ages = '*'
time = [f'{år}K1' for år in range(2008, 2011)]

comma_char = '%2C' # separation character for making url

request_options =  {'OMRÅDE':area,
                    'TID': comma_char.join(map(str, time)),
                    'ALDER': ages}
    
dst_data_kom = dst_api_dataframe('FOLK1A', options_input=request_options)

https://api.statbank.dk/v1/data/FOLK1A/csv?valuePresentation=Default&timeOrder=Ascending&allowVariablesInHead=true&OMRÅDE=*&TID=2008K1%2C2009K1%2C2010K1&ALDER=*


Basic structuring of the dataset which includes renaming to english names. We rename `OMRÅDE`, which means area, as it contains municipality names .

In [100]:
remove = (dst_data_kom.OMRÅDE=='Hele landet') | \
         dst_data_kom.OMRÅDE.str.contains('Region')

dst_data_kom = dst_data_kom.loc[~remove].copy()

dst_data_kom['age'] = \
    dst_data_kom.ALDER.str.findall("[0-9]*").str[0].replace('',pd.np.nan).astype(float)

dst_data_kom.drop('ALDER', axis=1, inplace=True)

dst_data_kom.rename(columns={'INDHOLD':'obs_value',
                             'OMRÅDE':'navn'}, 
                    inplace=True)
dst_data_kom['year'] = dst_data_kom.TID.str[:4].astype(int)

***Structuring the data***

We compute the share of young people in each municipality.

In [101]:
# define youth variable
dst_data_kom['young'] = dst_data_kom.age.between(20,29)
c_share = 'Share of young (20-30)'
c_count = 'Count of young (20-30)'

# count number of young using groupby
young_count_mun = dst_data_kom\
                .groupby(['navn', 'year', 'young'])\
                .obs_value.sum()\
                .unstack(level=2)\
        

young_share_mun = young_count_mun\
                .pipe(lambda df: df[True]/df.sum(1))\
                .rename(c_share)\
    
    
youth_stats_mun = pd.concat([young_share_mun, 
                         young_count_mun[True].rename(c_count)], 1)\
                .pipe(lambda df: df.mask(df.isnull().max(1)).reset_index())